In [16]:
import csv
import cv2
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split

In [17]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda,Dropout,Cropping2D
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D

In [18]:
# Change to locate path and Read image 
def read_image(source_path):
    filename = source_path.split('/')[-1]
    current_path = '/home/yue/CarND-Behavioral-Cloning/Training_data2/IMG/' + filename
    image = cv2.imread(current_path)
    converted_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return converted_image


# Open csv file
lines = []
with open('/home/yue/CarND-Behavioral-Cloning/Training_data2/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

In [19]:
# split dataset to 80% training data, 20% validation data
train_samples, validation_samples = train_test_split(lines, test_size = 0.2)

In [20]:
def brightness_image(image):
    image_tmp = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    image_tmp = np.array(image_tmp, dtype = np.float64)
    random_bright = .5+np.random.uniform()
    image_tmp[:,:,2] = image_tmp[:,:,2]*random_bright
    image_tmp[:,:,2][image_tmp[:,:,2]>255]  = 255
    image_tmp = np.array(image_tmp, dtype = np.uint8)
    image_tmp = cv2.cvtColor(image_tmp,cv2.COLOR_HSV2RGB)
    
    return image_tmp


In [21]:
def data_augmentation(image,steer, aug_num):
    images = []
    measurements = [] 
    images.append(image)
    measurements.append(steer)
    flip_image = np.fliplr(image)
    images.append(flip_image)
    measurements.append(-steer)
    
    for i in range(aug_num):
        b_image = brightness_image(image)
        b_flip_image = brightness_image(flip_image)
        images.append(b_image)
        measurements.append(steer)
        images.append(b_flip_image)
        measurements.append(-steer)
    
    return images, measurements
    
    
    

In [22]:
test_image = read_image(train_samples[0][0])
print(test_image.shape)
test_images, test_measurements = data_augmentation(test_image, 0.2, 1)
print(len(test_images))
print(len(test_measurements))

(160, 320, 3)
4
4


In [23]:
# Generator with coroutine
def train_generator(samples, batch_size = 32):
    num_samples = len(samples)
    while 1:
        # shuffle data
        sklearn.utils.shuffle(samples)
        #Loop over batches
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            measurements = []
            for batch_sample in batch_samples:
                # read camera images
                image_center = read_image(batch_sample[0])
                image_left = read_image(batch_sample[1])
                image_right = read_image(batch_sample[2])

                #images.append(image_center)
                #images.append(image_left)
                #images.append(image_right)
                
                # left and right camera correctness
                
                correction = 0.15
                steering_center = float(batch_sample[3])
                steering_left = steering_center + correction
                steering_right = steering_center - correction
                center_cluster = data_augmentation(image_center,steering_center,1)
                left_cluster = data_augmentation(image_left,steering_left,1)
                right_cluster = data_augmentation(image_right,steering_right,1)
                images = images + center_cluster[0] + left_cluster[0] + right_cluster[0]
                
                measurements = measurements + center_cluster[1] + left_cluster[1] + right_cluster[1]
                
                
                '''
                correction = 0.1
                steering_center = float(batch_sample[3])
                steering_left = steering_center + correction
                steering_right = steering_center - correction
                
                measurements.append(steering_center)
                measurements.append(steering_left)
                measurements.append(steering_right)
                '''
            X_train = np.array(images)
            y_train = np.array(measurements)
            yield sklearn.utils.shuffle(X_train, y_train)

In [24]:
# Generator with coroutine
def valid_generator(samples, batch_size = 32):
    num_samples = len(samples)
    while 1:
        # shuffle data
        sklearn.utils.shuffle(samples)
        #Loop over batches
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            measurements = []
            for batch_sample in batch_samples:
                # read camera images
                image_center = read_image(batch_sample[0])

                images.append(image_center)
                
                steering = float(batch_sample[3])
                
                measurements.append(steering)

            X_train = np.array(images)
            y_train = np.array(measurements)
            yield sklearn.utils.shuffle(X_train, y_train)

In [25]:
train_generator = train_generator(train_samples,batch_size=32)
validation_generator = valid_generator(validation_samples, batch_size=32)


In [26]:
model = Sequential()
#Normalize the data
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))
#Crop images to 80X320X3
model.add(Cropping2D(cropping=((60,20),(0,0))))
#Nvidia Network
model.add(Convolution2D(24,5,5,subsample=(2,2),activation = 'relu'))
model.add(Convolution2D(36,5,5,subsample=(2,2),activation = 'relu'))
model.add(Convolution2D(48,3,3,subsample=(2,2),activation = 'relu'))
model.add(Convolution2D(64,3,3,subsample=(1,1),activation = 'relu'))
model.add(Convolution2D(64,3,3,subsample=(1,1),activation = 'relu'))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(100, activation = 'relu'))
model.add(Dense(50, activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(1))

#Optimize MSE using ADAM optimizer
model.compile(loss='mse', optimizer='adam')
train_steps = np.ceil(len(train_samples)/32).astype(np.int32)
validation_steps =np.ceil(len(validation_samples)/32).astype(np.int32)

model.fit_generator(train_generator, samples_per_epoch=len(train_samples), validation_data=validation_generator, nb_val_samples=len(validation_samples), nb_epoch = 5)


model.save('model.h5')

/home/yue/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(24, (5, 5), activation="relu", strides=(2, 2))`
  import sys
/home/yue/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(36, (5, 5), activation="relu", strides=(2, 2))`
  
/home/yue/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (3, 3), activation="relu", strides=(2, 2))`
  if __name__ == '__main__':
/home/yue/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", strides=(1, 1))`
  # Remove the CWD from sys.path while we load stuff.
/home/yue/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launc

Epoch 1/5


UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node conv2d_26/convolution (defined at /home/yue/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:3185) ]]
	 [[node loss_5/mul (defined at /home/yue/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/keras/engine/training.py:865) ]]

Caused by op 'conv2d_26/convolution', defined at:
  File "/home/yue/anaconda3/envs/carnd-term1/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/yue/anaconda3/envs/carnd-term1/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/yue/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/yue/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/traitlets/config/applicati